# 8F_controls

08/25/2020
- positive controls = common loops, common ATAC, and expressed genes across all types
- negative controls = no loop, no ATAC, no expression across all cell types 







In [59]:
from collections import Counter, defaultdict
import pandas as pd
import numpy as np; np.random.seed(0)
import os, glob
import seaborn as sns; sns.set()
import matplotlib
import matplotlib.pyplot as plt
import pybedtools
from scipy.stats import ttest_ind

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
save_dir = '../data/processed/fig4_modelling/vocab_sum_final/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [4]:
data_all = pd.read_csv('/Users/mguo123/Google Drive/1_khavari/omics_project-LD/pan_omics/data/processed/tissue_crms/all_count_sep_overall.csv',index_col=0,header=0)


In [5]:
print(len(data_all.tissue.unique()))
data_all.tissue.unique()

32


array(['A431-CTRLi', 'A431-p63i', 'Airway', 'Astrocytes', 'Bladder',
       'CAL27-CTRLi', 'CAL27-p63i', 'COLO_SCR_DMSO', 'COLO_SCR_PLX',
       'COLO_shMITF_DMSO', 'COLO_shMITF_PLX', 'Colon', 'D0-CTRLi',
       'D0-p63i', 'D3-CTRLi', 'D3-p63i', 'Esophageal', 'GDSD0', 'GDSD3',
       'GDSD6', 'GM12878', 'HMEC', 'Melanocytes', 'Ovarian', 'Pancreas',
       'Prostate', 'Renal', 'SCC13-CTRLi', 'SCC13-p63i', 'Thyroid',
       'Uterine', 'WM_SCR_DMSO'], dtype=object)

In [7]:
# # expression labels
# exp_label = list(np.log10(data_all.exp.values+1e-2))
# labels_all  = np.array(np.array(exp_label)>THRES)

# tissues_label  = data_all.tissue.values#np.array((data_all.exp>THRES).values)

# genes_all = data_all.index.values


# negative controls
no loop, no ATAC, no expression across all cell types for negative controls

for the negative controls, narrow down the list by loci that don’t have an ATAC peak within a certain distance: Like 100 KB or some such? I

make sure actually in a gene desert or region of inaccessible chromatin so really should be “off” / negative


In [20]:
NEG_DIST_MIN = 25000 # for negative control, gene must by at least 10kb away from closest atac peak

In [21]:
%%time
neg_crms = data_all[data_all.iloc[:,1:].sum(axis=1)<0.001]

CPU times: user 21.7 s, sys: 7.43 s, total: 29.1 s
Wall time: 29.2 s


In [22]:
print(data_all.shape)
print(neg_crms.shape)

(793952, 1593)
(227421, 1593)


In [23]:
gene_tissue_counts = pd.Series(neg_crms.index).value_counts()
neg_genes = gene_tissue_counts[gene_tissue_counts==len(data_all.tissue.unique())].index
len(neg_genes) # number of genes with no loop, ATAC or expression (looped or proximal)

2502

In [24]:
gene_regions = pybedtools.BedTool('../data/external/gencode.v19.gene.bed').to_dataframe()
gene_regions_neg = gene_regions[gene_regions.name.isin(neg_genes)]
print(gene_regions_neg.shape) # # genes with genomic location
gene_regions_neg_bed = pybedtools.BedTool.from_dataframe(gene_regions_neg).sort()

(1809, 4)


In [25]:
print(len(glob.glob('../data/interim/merged/atac/*bed')))
sorted(glob.glob('../data/interim/merged/atac/*bed'))

35


['../data/interim/merged/atac/A431-CTRLi_merged.bed',
 '../data/interim/merged/atac/A431-p63i_merged.bed',
 '../data/interim/merged/atac/Airway_merged.bed',
 '../data/interim/merged/atac/Astrocytes_merged.bed',
 '../data/interim/merged/atac/Bladder_merged.bed',
 '../data/interim/merged/atac/CAL27-CTRLi_merged.bed',
 '../data/interim/merged/atac/CAL27-p63i_merged.bed',
 '../data/interim/merged/atac/COLO_SCR_DMSO_merged.bed',
 '../data/interim/merged/atac/COLO_SCR_PLX_merged.bed',
 '../data/interim/merged/atac/COLO_shMITF_DMSO_merged.bed',
 '../data/interim/merged/atac/COLO_shMITF_PLX_merged.bed',
 '../data/interim/merged/atac/Colon_merged.bed',
 '../data/interim/merged/atac/D0-CTRLi_merged.bed',
 '../data/interim/merged/atac/D0-p63i_merged.bed',
 '../data/interim/merged/atac/D3-CTRLi_merged.bed',
 '../data/interim/merged/atac/D3-p63i_merged.bed',
 '../data/interim/merged/atac/D6-CTRLi_merged.bed',
 '../data/interim/merged/atac/D6-p63i_merged.bed',
 '../data/interim/merged/atac/Esophagea

In [26]:
%%time
all_far_genes = []
all_far_genes_dict = {}
for atac_bed in sorted(glob.glob('../data/interim/merged/atac/*bed')):
    tissue = os.path.basename(atac_bed).split('_merged')[0]
    print(tissue)
    gene_regions_neg_bed_tissue = gene_regions_neg_bed.closest(atac_bed,d=True)

    gene_dist_df = gene_regions_neg_bed_tissue.to_dataframe().groupby('name').agg({'thickEnd':min})
    gene_dist_df.columns = ['dist']
    far_genes_tissue = list(gene_dist_df.index[gene_dist_df.dist > NEG_DIST_MIN])
    print(len(far_genes_tissue))
    all_far_genes+=far_genes_tissue
    all_far_genes_dict[tissue] = far_genes_tissue

A431-CTRLi
844
A431-p63i
840
Airway
727
Astrocytes
1226
Bladder
690
CAL27-CTRLi
722
CAL27-p63i
648
COLO_SCR_DMSO
646
COLO_SCR_PLX
663
COLO_shMITF_DMSO
610
COLO_shMITF_PLX
656
Colon
796
D0-CTRLi
993
D0-p63i
896
D3-CTRLi
925
D3-p63i
858
D6-CTRLi
1062
D6-p63i
1063
Esophageal
731
GDSD0
769
GDSD3
745
GDSD6
739
GM12878
897
HMEC
735
Melanocytes
1151
Ovarian
1132
Pancreas
731
Prostate
1040
Renal
776
SCC13-CTRLi
664
SCC13-p63i
634
SKMEL5_SCR_DMSO
579
Thyroid
801
Uterine
688
WM_SCR_DMSO
678
CPU times: user 410 ms, sys: 361 ms, total: 770 ms
Wall time: 4.47 s


In [27]:
far_genes_count = pd.Series(Counter(all_far_genes))
neg_genes_far = sorted(far_genes_count.index[far_genes_count==len(glob.glob('../data/interim/merged/atac/*bed'))])
print(len(neg_genes_far))

275


In [28]:
print(neg_genes_far)

['BIRC8', 'CDY1', 'CDY2A', 'CSPG4P1Y', 'CT47A12', 'CT47A6', 'DAZ1', 'DAZ3', 'DAZ4', 'DEFB133', 'FAM197Y2', 'FAM197Y3', 'FAM197Y7', 'FAM197Y8', 'FAM197Y9', 'FAM25E', 'FAM74A1', 'FAM90A27P', 'FOXR2', 'GAGE12C', 'GAGE12D', 'GAGE2C', 'GAGE2E', 'GOLGA2P2Y', 'GOLGA8DP', 'GS1-278J22.2', 'KIR3DL3', 'LINC00280', 'LINC00408', 'LINC00442', 'MAGEB5', 'MIR105-1', 'MIR105-2', 'MIR1251', 'MIR1267', 'MIR1283-1', 'MIR1295A', 'MIR1298', 'MIR147A', 'MIR3118-1', 'MIR3118-2', 'MIR3118-4', 'MIR3169', 'MIR3668', 'MIR3683', 'MIR378C', 'MIR383', 'MIR3914-1', 'MIR4273', 'MIR4283-1', 'MIR4283-2', 'MIR4431', 'MIR4445', 'MIR4460', 'MIR4509-2', 'MIR4509-3', 'MIR4527', 'MIR4643', 'MIR4675', 'MIR4770', 'MIR4791', 'MIR513A1', 'MIR513B', 'MIR513C', 'MIR514A1', 'MIR514A2', 'MIR514A3', 'MIR517A', 'MIR517B', 'MIR518B', 'MIR518F', 'MIR519B', 'MIR519D', 'MIR520A', 'MIR520B', 'MIR520C', 'MIR520D', 'MIR521-2', 'MIR525', 'MIR526A1', 'MIR526B', 'MIR548AB', 'MIR548F1', 'MIR548F3', 'MIR548F4', 'MIR548F5', 'MIR548I1', 'MIR548I2', 

In [29]:
pd.Series(neg_genes_far).to_csv(os.path.join(save_dir, 'neg_genes_far.csv'),index=None,header=None)

/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


# 2. positive controls

positive controls = common loops, common ATAC, and expressed genes across all types

In [6]:
THRES=1 # for RNA
normal_tissues_pos = [ 'Airway', 'Astrocytes', 'Bladder', 'Colon', 
         'Esophageal', 'GDSD0', 'GDSD3', 'GDSD6', 'GM12878', 'HMEC', 'Melanocytes', 'Ovarian', 'Pancreas',
       'Prostate', 'Renal', 'Thyroid',       'Uterine']
cancer_tissues_pos = ['A431-CTRLi', 'CAL27-CTRLi', 'SCC13-CTRLi','COLO_SCR_DMSO','WM_SCR_DMSO',
                     'A431-p63i','CAL27-p63i', 'SCC13-p63i', 'COLO_SCR_PLX','COLO_shMITF_DMSO', 'COLO_shMITF_PLX']

all_tissue_pos = normal_tissues_pos + ['A431-CTRLi', 'CAL27-CTRLi', 'SCC13-CTRLi','COLO_SCR_DMSO','WM_SCR_DMSO']
#                'D0-CTRLi','D3-CTRLi', 'SCC13-p63i', 'D0-p63i', 'D3-p63i',
#                'A431-CTRLi', 'CAL27-CTRLi',  'SCC13-CTRLi','COLO_SCR_DMSO','WM_SCR_DMSO']

In [7]:
# expressed_genes

expr_df = data_all[data_all.tissue.isin(all_tissue_pos)]['exp'].reset_index()
expr_df.columns = ['gene','exp']
expr_df_min = expr_df.groupby('gene').agg({'exp':'min'}) # basically find lowest expressed gene 
expr_genes = expr_df_min.index[expr_df_min.exp>THRES]
print(len(expr_genes)) # number of commonly expressed genes

#common ATAC
atac_pro_df = data_all[data_all.tissue.isin(all_tissue_pos)].num_atac_regions_pro.reset_index()
atac_pro_df.columns = ['gene','atac']
atac_pro_df_min = atac_pro_df.groupby('gene').agg({'atac':'min'}) # basically find lowest # of atac peaks in the promoter region gene 
atac_genes = atac_pro_df_min.index[atac_pro_df_min.atac>0]
print(len(atac_genes)) # number of atac genes that have at least 1 atac peak in promoter region (2kb upstream, 500 bp downstream of tss) in all tissues

# common loop
hichip_pro_df = data_all[data_all.tissue.isin(all_tissue_pos)].num_loops.reset_index()
hichip_pro_df.columns = ['gene','hichip']
hichip_pro_df_min = hichip_pro_df.groupby('gene').agg({'hichip':'min'}) # basically find lowest # of atac peaks in the promoter region gene 
hichip_genes = hichip_pro_df_min.index[hichip_pro_df_min.hichip>0]
print(len(hichip_genes)) # number of atac genes that have at least 1 loop to promoter region (2kb upstream, 500 bp downstream of tss) in all tissues

all_pos_genes = sorted(set(expr_genes).intersection(set(atac_genes)).intersection(set(hichip_genes)))
len(all_pos_genes)

6642
468
1780


85

In [8]:
expr_genes

Index(['AAAS', 'AACS', 'AAGAB', 'AAK1', 'AAMDC', 'AAMP', 'AAR2', 'AARS1',
       'AARSD1', 'AASDH',
       ...
       'ZSWIM1', 'ZSWIM7', 'ZSWIM8', 'ZUP1', 'ZW10', 'ZWILCH', 'ZWINT',
       'ZYG11B', 'ZYX', 'ZZZ3'],
      dtype='object', name='gene', length=6642)

In [ ]:
all_pos_genes

In [161]:
pd.Series(all_pos_genes).to_csv(os.path.join(save_dir, 'all_pos_genes.csv'),index=None,header=None)

/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [135]:
# expressed_genes

expr_df = data_all[data_all.tissue.isin(normal_tissues_pos)]['exp'].reset_index()
expr_df.columns = ['gene','exp']
expr_df_min = expr_df.groupby('gene').agg({'exp':'min'}) # basically find lowest expressed gene 
expr_genes = expr_df_min.index[expr_df_min.exp>THRES]
print(len(expr_genes)) # number of commonly expressed genes

#common ATAC
atac_pro_df = data_all[data_all.tissue.isin(normal_tissues_pos)].num_atac_regions_pro.reset_index()
atac_pro_df.columns = ['gene','atac']
atac_pro_df_min = atac_pro_df.groupby('gene').agg({'atac':'min'}) # basically find lowest # of atac peaks in the promoter region gene 
atac_genes = atac_pro_df_min.index[atac_pro_df_min.atac>0]
print(len(atac_genes)) # number of atac genes that have at least 1 atac peak in promoter region (2kb upstream, 500 bp downstream of tss) in all tissues

# common loop
hichip_pro_df = data_all[data_all.tissue.isin(normal_tissues_pos)].num_loops.reset_index()
hichip_pro_df.columns = ['gene','hichip']
hichip_pro_df_min = hichip_pro_df.groupby('gene').agg({'hichip':'min'}) # basically find lowest # of atac peaks in the promoter region gene 
hichip_genes = hichip_pro_df_min.index[hichip_pro_df_min.hichip>0]
print(len(hichip_genes)) # number of atac genes that have at least 1 loop to promoter region (2kb upstream, 500 bp downstream of tss) in all tissues

normal_pos_genes = list(set(expr_genes).intersection(set(atac_genes)).intersection(set(hichip_genes)))
len(normal_pos_genes)

7091
492
2034


97

In [136]:
# expressed_genes

expr_df = data_all[data_all.tissue.isin(cancer_tissues_pos)]['exp'].reset_index()
expr_df.columns = ['gene','exp']
expr_df_min = expr_df.groupby('gene').agg({'exp':'min'}) # basically find lowest expressed gene 
expr_genes = expr_df_min.index[expr_df_min.exp>THRES]
print(len(expr_genes)) # number of commonly expressed genes

#common ATAC
atac_pro_df = data_all[data_all.tissue.isin(cancer_tissues_pos)].num_atac_regions_pro.reset_index()
atac_pro_df.columns = ['gene','atac']
atac_pro_df_min = atac_pro_df.groupby('gene').agg({'atac':'min'}) # basically find lowest # of atac peaks in the promoter region gene 
atac_genes = atac_pro_df_min.index[atac_pro_df_min.atac>0]
print(len(atac_genes)) # number of atac genes that have at least 1 atac peak in promoter region (2kb upstream, 500 bp downstream of tss) in all tissues

# common loop
hichip_pro_df = data_all[data_all.tissue.isin(cancer_tissues_pos)].num_loops.reset_index()
hichip_pro_df.columns = ['gene','hichip']
hichip_pro_df_min = hichip_pro_df.groupby('gene').agg({'hichip':'min'}) # basically find lowest # of atac peaks in the promoter region gene 
hichip_genes = hichip_pro_df_min.index[hichip_pro_df_min.hichip>0]
print(len(hichip_genes)) # number of atac genes that have at least 1 loop to promoter region (2kb upstream, 500 bp downstream of tss) in all tissues


cancer_pos_genes = list(set(expr_genes).intersection(set(atac_genes)).intersection(set(hichip_genes)))
len(cancer_pos_genes)

7837
922
3336


267

In [137]:
len(set(normal_pos_genes).union(set(cancer_pos_genes)))

289

In [138]:
pos_genes_df = pd.concat([pd.DataFrame({'gene':normal_pos_genes,'type':'normal'}),
           pd.DataFrame({'gene':cancer_pos_genes,'type':'cancer'})],axis=0)
pos_genes_df = pos_genes_df.groupby('gene').agg({'type':'|'.join }).reset_index()
pos_genes_df.to_csv(os.path.join(save_dir, 'pos_genes_df.csv'))

In [139]:
pos_genes_df

,gene,type
0,ABCB6,normal
1,ABCF3,normal|cancer
2,ABHD17A,cancer
3,ACADS,cancer
4,ACADVL,normal
...,...,...
284,ZNF513,cancer
285,ZNF580,normal|cancer
286,ZNF581,normal|cancer
287,ZNF771,cancer


# 3. Check Controls for Motifs (MOODS

09/09/2020

after 8F1_make_mpra


In [103]:
from scipy.stats import ttest_ind
import pandas as pd
import numpy as np; np.random.seed(0)
import os, glob
import seaborn as sns; sns.set()
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter

In [14]:
glob.glob('../data/processed/fig4_modelling/vocab_sum_final/mpra*')

['../data/processed/fig4_modelling/vocab_sum_final/mpra_oligo_df_final_SUBMIT.txt',
 '../data/processed/fig4_modelling/vocab_sum_final/mpra_vocab_seq2.fasta',
 '../data/processed/fig4_modelling/vocab_sum_final/mpra_control_seq.fasta',
 '../data/processed/fig4_modelling/vocab_sum_final/mpra_lib_seq.csv',
 '../data/processed/fig4_modelling/vocab_sum_final/mpra_vocab_seq.fasta',
 '../data/processed/fig4_modelling/vocab_sum_final/mpra_oligo_df_final.txt',
 '../data/processed/fig4_modelling/vocab_sum_final/mpra_oligo_df_filt.csv']

In [109]:
# read rna
rna_df = pd.read_csv('../data/interim/rna/tissue_tpm_sym.csv',index_col=0)
print(rna_df.columns)

Index(['A431-CTRLi', 'A431-p63i', 'Airway', 'Astrocytes', 'Bladder',
       'CAL27-CTRLi', 'CAL27-p63i', 'COLO_SCR_DMSO', 'COLO_SCR_PLX',
       'COLO_shMITF_DMSO', 'COLO_shMITF_PLX', 'Colon', 'D0-CTRLi', 'D0-p63i',
       'D3-CTRLi', 'D3-p63i', 'Esophageal', 'GDSD0', 'GDSD3', 'GDSD6',
       'GM12878', 'HMEC', 'Melanocytes', 'Ovarian', 'Pancreas', 'Prostate',
       'Renal', 'SCC13-CTRLi', 'SCC13-p63i', 'SKMEL5_SCR_DMSO',
       'SKMEL5_SCR_PLX', 'SKMEL5_shMITF-DMSO', 'SKMEL5_shMITF_PLX', 'Thyroid',
       'Uterine', 'WM_SCR_DMSO', 'WM_SCR_PLX', 'WM_shMITF_DMSO',
       'WM_shMITF_PLX'],
      dtype='object')


In [19]:
# read mpra info 
mpra_df = pd.read_csv( '../data/processed/fig4_modelling/vocab_sum_final/mpra_oligo_df_final.txt', sep='\t')
name_to_seq_type = pd.Series(mpra_df.seq_type.values, index=mpra_df.name.values).to_dict()
# want seq_type 'pos' or 'neg'
len(name_to_seq_type)

9721

In [132]:
# check rna tpm values of the pos controls 
mpra_df_controls = mpra_df[mpra_df.seq_type.isin(['pos','neg'])]
mpra_df_controls_instance = mpra_df_controls[['name','seq_type']].drop_duplicates().reset_index(drop=True)
print(mpra_df_controls_instance.shape)
mpra_df_controls_instance['Colon_exp'] = mpra_df_controls_instance.name.apply(lambda x: rna_df.loc[x, 'Colon'])
mpra_df_controls_instance['GDSD6_exp'] = mpra_df_controls_instance.name.apply(lambda x: rna_df.loc[x, 'GDSD6'])
mpra_df_controls_instance['GM12878_exp'] = mpra_df_controls_instance.name.apply(lambda x: rna_df.loc[x, 'GM12878'])
mpra_df_controls_instance['Melanocytes_exp'] = mpra_df_controls_instance.name.apply(lambda x: rna_df.loc[x, 'Melanocytes'])
mpra_df_controls_instance['A431-CTRLi_exp'] = mpra_df_controls_instance.name.apply(lambda x: rna_df.loc[x, 'A431-CTRLi'])
mpra_df_controls_instance['CAL27-CTRLi_exp'] = mpra_df_controls_instance.name.apply(lambda x: rna_df.loc[x, 'CAL27-CTRLi'])
mpra_df_controls_instance['SCC13-CTRLi_exp'] = mpra_df_controls_instance.name.apply(lambda x: rna_df.loc[x, 'SCC13-CTRLi'])
mpra_df_controls_instance['COLO_SCR_DMSO_exp'] = mpra_df_controls_instance.name.apply(lambda x: rna_df.loc[x, 'COLO_SCR_DMSO'])
mpra_df_controls_instance['WM_SCR_DMSO_exp'] = mpra_df_controls_instance.name.apply(lambda x: rna_df.loc[x, 'WM_SCR_DMSO'])
mpra_df_controls_instance.sort_values(['seq_type','name'],inplace=True)
mpra_df_pos_controls_instance = mpra_df_controls_instance[mpra_df_controls_instance.seq_type=='pos'].reset_index(drop=True)
display(mpra_df_pos_controls_instance.describe())

(161, 2)


,Colon_exp,GDSD6_exp,GM12878_exp,Melanocytes_exp,A431-CTRLi_exp,CAL27-CTRLi_exp,SCC13-CTRLi_exp,COLO_SCR_DMSO_exp,WM_SCR_DMSO_exp
count,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000
mean,191.039861,184.404097,170.757569,181.170903,230.207153,195.343333,186.955347,175.035139,150.917639
std,1028.035268,1123.249095,1000.823725,953.065999,1554.291379,1106.646907,1044.275578,1067.553131,788.453088
min,1.295000,1.275000,1.010000,2.085000,1.635000,1.650000,2.000000,2.235000,2.180000
25%,6.657500,4.837500,4.705000,8.926250,7.358750,7.555000,8.231250,7.828750,8.657500
50%,13.827500,12.442500,10.917500,17.052500,18.065000,15.692500,17.017500,17.302500,16.397500
75%,34.031250,34.037500,29.716250,48.647500,45.266250,44.555000,52.770000,48.397500,50.161250
max,8659.390000,9478.325000,8414.700000,7994.435000,13203.895000,9324.900000,8843.750000,9058.270000,6662.385000


In [106]:
name_to_seq_type_count = Counter(name_to_seq_type.values())
name_to_seq_type_count

Counter({'pos': 72, 'vocab': 9560, 'neg': 89})

In [20]:
#read tf info
tf_annon_df = pd.read_csv('../data/external/HOCOMOCOv11_annotation.csv',index_col=0)
tf_annon_df['id_trim'] = tf_annon_df['id'] + '.pwm.trim'
tf_name_to_id_dict = pd.Series(tf_annon_df.id_trim.values, index=tf_annon_df.tf.values).to_dict()
tf_id_to_name_dict = pd.Series(tf_annon_df.tf.values, index=tf_annon_df.id_trim.values).to_dict()
tf_name_to_id_abbr_dict = pd.Series(tf_annon_df.id.values, index=tf_annon_df.tf.values).to_dict()
tf_id_abbr_to_name_dict = pd.Series(tf_annon_df.tf.values, index=tf_annon_df.id.values).to_dict()


In [217]:
# motif_scan_control_dir = '../data/processed/fig4_modelling/vocab_sum_final/motif_scan_control/'
motif_scan_control_dir = '../data/processed/fig4_modelling/vocab_sum_final/motif_scan_control_pval001/'
motif_scan_control_files = glob.glob(os.path.join(motif_scan_control_dir, '*.csv'))
print(len(motif_scan_control_files))

769


In [134]:
def get_str_arr(l):
    return '|'.join([str(x) for x in l])

def ttest_motif(row):
    list_neg = [float(x) for x in row['score_get_str_arr_neg'].split('|')]
    list_pos = [float(x) for x in row['score_get_str_arr_pos'].split('|')]
    t, p = ttest_ind(list_pos, list_neg)
    return t,p

In [210]:
%%time
pos_neg_scan_info_all=pd.DataFrame()
pos_neg_instance_scan_info_all=pd.DataFrame()

# for motif_scan_control_file in motif_scan_control_files[:5]: ## DEBUG
for motif_scan_control_file in motif_scan_control_files:

    motif_scan_results = pd.read_csv(motif_scan_control_file,names=['name_config','pwm_file','pos','strand','score','seq','seq_actual'])
    motif = tf_id_abbr_to_name_dict[os.path.basename(motif_scan_control_file).split('_scan')[0]]
    motif_scan_results['motif'] = motif_scan_results.pwm_file.str.split('.pwm',expand=True).iloc[:,0].map(tf_id_abbr_to_name_dict)
    motif_scan_results['seq_type']  = motif_scan_results.name_config.map(name_to_seq_type)
    
    # create  df, name_config,seq_type motif, avg score
    instance_scan_info = motif_scan_results.groupby(['name_config','seq_type','motif']).agg({'score':['count','mean','std','min','max']})
    instance_scan_info.columns = ['_'.join(x) for x in instance_scan_info.columns ]
    instance_scan_info.reset_index(inplace=True)
    pos_neg_instance_scan_info_all = pd.concat([pos_neg_instance_scan_info_all, instance_scan_info],axis=0)

    # # create  df, seq_type motif, avg score
    pos_neg_scan_info = motif_scan_results.groupby(['seq_type','motif']).agg({'score':['count','mean','std','min','max', get_str_arr ]})
    pos_neg_scan_info.columns = ['_'.join(x) for x in pos_neg_scan_info.columns ]
    pos_neg_scan_info.reset_index(inplace=True)
    pos_neg_scan_info_all = pd.concat([pos_neg_scan_info_all, pos_neg_scan_info],axis=0)
    


CPU times: user 23.9 s, sys: 99.5 ms, total: 24 s
Wall time: 24 s


### check to see which motifs are enriched in controls, and do a t test to compare pos vs neg (MOODS)

In [136]:
# there are duplicate motif values
print(pos_neg_scan_info_all.shape)
pos_neg_scan_info_all = pos_neg_scan_info_all.groupby(['seq_type','motif']).agg({'score_count':sum,
                                                        'score_mean': 'mean',
                                                        'score_std': 'mean',
                                                         'score_min': min,
                                                         'score_max':max,
                                                         'score_get_str_arr': lambda x: '|'.join(x)
                                                        }).reset_index()
print(pos_neg_scan_info_all.shape)


(1538, 8)
(1356, 8)


In [137]:
pos_neg_scan_info_wide = pos_neg_scan_info_all.pivot(index='motif',columns='seq_type',values=['score_count','score_mean','score_std','score_min','score_max','score_get_str_arr'])
pos_neg_scan_info_wide.columns = ['_'.join(x) for x in pos_neg_scan_info_wide.columns ]
pos_neg_scan_info_wide['mean_pos_neg_range'] = pos_neg_scan_info_wide.score_mean_pos - pos_neg_scan_info_wide.score_mean_neg
pos_neg_scan_info_wide = pd.concat([pos_neg_scan_info_wide, pos_neg_scan_info_wide.apply(ttest_motif,axis=1).apply(lambda s: pd.Series({'t_val':s[0], 'p_val':s[1]}))],axis=1)
pos_neg_scan_info_wide['p_val_bonf'] = pos_neg_scan_info_wide.p_val.apply(lambda x: min (1, x*pos_neg_scan_info_wide.shape[0]))

/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3506: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [138]:
print(pos_neg_scan_info_wide.shape)
pos_neg_scan_info_wide_filt = pos_neg_scan_info_wide[pos_neg_scan_info_wide.p_val_bonf<0.05]
print(pos_neg_scan_info_wide_filt.shape)

(678, 16)
(64, 16)


In [139]:
motif_count_pos_neg_ratio = pd.Series(pos_neg_scan_info_wide.score_count_pos/pos_neg_scan_info_wide.score_count_neg*name_to_seq_type_count['neg']/name_to_seq_type_count['pos']).sort_values()
print(motif_count_pos_neg_ratio.shape)
motif_count_pos_neg_ratio[motif_count_pos_neg_ratio>1].shape, motif_count_pos_neg_ratio[motif_count_pos_neg_ratio<1].shape


(678,)


((407,), (271,))

In [140]:
pos_neg_scan_info_wide_filt[pos_neg_scan_info_wide_filt.t_val<0].index.values

array(['ARID5B', 'BCL6', 'DBP', 'EMX2', 'GATA3', 'GFI1B', 'HNF1B',
       'HOXB4', 'HOXC13', 'KLF8', 'LHX2', 'MEIS1', 'MIXL1', 'MSX2',
       'MYNN', 'NKX6-1', 'NOBOX', 'NR2E1', 'NR2E3', 'PDX1', 'POU3F1',
       'POU5F1B', 'PRDM6', 'SIX1', 'SMARCA1', 'SOX10', 'STAT1', 'STAT2',
       'STAT5A', 'STAT5B', 'STAT6', 'TEAD3', 'TEF', 'TWIST1', 'VAX2',
       'ZBTB49', 'ZNF232', 'ZNF274'], dtype=object)

In [141]:
pos_neg_scan_info_wide_filt[pos_neg_scan_info_wide_filt.t_val>0].index.values

array(['CTCF', 'CTCFL', 'EGR4', 'ETV1', 'FIGLA', 'KLF15', 'MAZ', 'NHLH1',
       'PLAG1', 'PLAGL1', 'PRRX1', 'SP1', 'SP2', 'SP4', 'THAP1', 'WT1',
       'ZBTB33', 'ZFP64', 'ZIC2', 'ZIC3', 'ZIC4', 'ZNF148', 'ZNF335',
       'ZNF467', 'ZNF563', 'ZNF589'], dtype=object)

In [142]:
pos_neg_scan_info_wide_filt[pos_neg_scan_info_wide_filt.t_val<0].index.values

array(['ARID5B', 'BCL6', 'DBP', 'EMX2', 'GATA3', 'GFI1B', 'HNF1B',
       'HOXB4', 'HOXC13', 'KLF8', 'LHX2', 'MEIS1', 'MIXL1', 'MSX2',
       'MYNN', 'NKX6-1', 'NOBOX', 'NR2E1', 'NR2E3', 'PDX1', 'POU3F1',
       'POU5F1B', 'PRDM6', 'SIX1', 'SMARCA1', 'SOX10', 'STAT1', 'STAT2',
       'STAT5A', 'STAT5B', 'STAT6', 'TEAD3', 'TEF', 'TWIST1', 'VAX2',
       'ZBTB49', 'ZNF232', 'ZNF274'], dtype=object)

## try to find vocab pairs within a single instance

In [204]:
vocab_pairs = mpra_df[mpra_df.seq_type=='vocab'].name.apply(lambda x:x.split('+')[0]).unique()
len(vocab_pairs)

239

In [211]:
pos_neg_instance_scan_info_all[:5]

,name_config,seq_type,motif,score_count,score_mean,score_std,score_min,score_max
0,ANAPC15,pos,ZNF85,1,1.316026,NaN,1.316026,1.316026
1,CELF1,pos,ZNF85,2,1.076643,0.808938,0.504637,1.648649
2,DEAF1,pos,ZNF85,1,2.637443,NaN,2.637443,2.637443
3,DEFB133,neg,ZNF85,1,5.499710,NaN,5.499710,5.499710
4,FOXR2,neg,ZNF85,1,8.319205,NaN,8.319205,8.319205


In [212]:
neg_instance_scan_info_all = pos_neg_instance_scan_info_all[pos_neg_instance_scan_info_all.seq_type=='neg']

In [215]:
neg_instance_motifs = neg_instance_scan_info_all.groupby('name_config').agg({'motif':'|'.join}).reset_index()

In [214]:
def check_vocab_pair(motif_str, vocab_pairs=vocab_pairs):
    motif_list = motif_str.split('|')
    vocab_pairs_found = []
    for vocab_pair in vocab_pairs:
        v1, v2 = vocab_pair.split('::')
        if (v1 in motif_list) & (v2 in motif_list):
            vocab_pairs_found.append(vocab_pair)
    return '|'.join(sorted(vocab_pairs_found)),len(vocab_pairs_found)

In [221]:
neg_instance_motifs = pd.concat([neg_instance_motifs,
                                 neg_instance_motifs.motif.apply(check_vocab_pair).apply(lambda s: pd.Series({'vocab_pairs_found':s[0], 'num_vocab_pairs_found':s[1]}))],axis=1)
neg_instance_motifs

,name_config,motif,vocab_pairs_found,num_vocab_pairs_found
0,BIRC8,TCF4|ZBTB33|RREB1|HIC2|FUBP1|ZNF320|HES5|KLF1|...,ARID5B::KLF5|ARNT::SP1|ASCL2::KLF5|BATF::KLF15...,35
1,CT47A12,ZNF589|MEF2C|THRA|ESRRB|ZNF143|ZNF341|E2F4|PAT...,ARID5B::KLF5|ARID5B::SOX10|ARID5B::STAT1|E2F1:...,109
2,CT47A6,ZNF589|MEF2C|THRA|ESRRB|ZNF143|ZNF341|E2F4|PAT...,ARID5B::KLF5|ARID5B::SOX10|ARID5B::STAT1|E2F1:...,109
3,DEFB133,ZNF85|THRA|ZIC2|ZNF768|ZNF467|FUBP1|ZNF320|FOX...,ARID5B::LEF1|ARID5B::SOX10|ARID5B::STAT1|CBFB:...,48
4,FAM25E,THRA|NANOG|ESRRB|HOXB4|ELK3|ARX|NFIB|ZNF768|TP...,BACH1::KLF5|BATF::KLF15|EBF1::ZNF816|EGR2::FOS...,62
...,...,...,...,...
84,USP17L30,ZNF589|ZNF143|HOXB4|ZIC2|ZNF708|TFE3|ZIC3|GCM1...,BATF::ZNF589|EGR2::FOSL2|EGR2::JUN|EGR2::JUNB|...,23
85,USP17L5,ZNF589|ZNF143|HOXB4|ZIC2|ZNF708|TFE3|ZIC3|GCM1...,BATF::ZNF589|EGR2::FOSL2|EGR2::JUN|EGR2::JUNB|...,23
86,ZNF479,GFI1|ZNF589|NANOG|ZBTB33|HOXB4|ZIC2|ETV6|E2F4|...,ARID5B::LEF1|ARID5B::SOX10|EGR1::IKZF1|EN2::PR...,24
87,ZNF679,E2F4|PATZ1|ZNF768|RUNX2|TP53|ZBED1|HIC2|TFE3|Z...,ARID5B::SOX10|E2F3::TP63|EGR1::IKZF1|EGR2::JUN...,33


In [225]:
neg_instance_motifs_counts = Counter('|'.join(list(neg_instance_motifs.vocab_pairs_found)).split('|'))

In [226]:
pos_instance_scan_info_all = pos_neg_instance_scan_info_all[pos_neg_instance_scan_info_all.seq_type=='pos']
pos_instance_motifs = pos_instance_scan_info_all.groupby('name_config').agg({'motif':'|'.join}).reset_index()
pos_instance_motifs = pd.concat([pos_instance_motifs,
                                 pos_instance_motifs.motif.apply(check_vocab_pair).apply(lambda s: pd.Series({'vocab_pairs_found':s[0], 'num_vocab_pairs_found':s[1]}))],axis=1)
pos_instance_motifs_counts = Counter('|'.join(list(pos_instance_motifs.vocab_pairs_found)).split('|'))

In [230]:
control_vocab_pair_dict = {}
for vocab in list(neg_instance_motifs_counts.keys()) + list(pos_instance_motifs_counts.keys()):
    control_vocab_pair_dict[vocab] = {'in_pos':vocab in pos_instance_motifs_counts,
                                      'in_neg':vocab in neg_instance_motifs_counts,
                                      'count_pos':pos_instance_motifs_counts.get(vocab, 0),
                                      'count_neg':neg_instance_motifs_counts.get(vocab, 0),
        
    }

In [235]:
control_vocab_pair_df = pd.DataFrame.from_dict(control_vocab_pair_dict,orient='index')
control_vocab_pair_df['pos_neg_ratio'] = (control_vocab_pair_df.count_pos+1)/(control_vocab_pair_df.count_neg+1)
control_vocab_pair_df['pos_neg_ratio_corr'] = control_vocab_pair_df.pos_neg_ratio*neg_instance_motifs.shape[0]/pos_instance_motifs.shape[0]
control_vocab_pair_df.sort_values('pos_neg_ratio_corr',inplace=True)
control_vocab_pair_df.describe()

,count_pos,count_neg,pos_neg_ratio,pos_neg_ratio_corr
count,239.000000,239.000000,239.000000,239.000000
mean,10.991632,13.460251,1.051737,1.300063
std,6.036299,9.753123,0.676504,0.836235
min,0.000000,0.000000,0.076923,0.095085
25%,6.000000,7.000000,0.618056,0.763985
50%,10.000000,11.000000,0.937500,1.158854
75%,15.000000,16.000000,1.292857,1.598115
max,28.000000,55.000000,4.500000,5.562500


# 4. AME enrichment

## AME motif enrichment for negative controls

`ame --verbose 1 --oc . --scoring avg --method fisher --hit-lo-fraction 0.25 --evalue-report-threshold 10.0 --control --shuffle-- --kmer 2 mpra_neg_control_seq.fasta motifs.meme`

AME motif enrichment neg control

motifs: '../data/external/hocomoco_human_trim_jaspar_format.txt'

In [145]:
save_dir

'../data/processed/fig4_modelling/vocab_sum_final/'

In [165]:
# num actual tfs used 
vocab_pairs = mpra_df[mpra_df.seq_type=='vocab'].name.apply(lambda x:x.split('+')[0]).unique()
list_tfs = []
for vocab_pair in vocab_pairs:
    vocab1, vocab2 = vocab_pair.split('::')
    list_tfs.append(vocab1)
    list_tfs.append(vocab2)
list_tfs = sorted(set(list_tfs))
len(list_tfs)

123

In [148]:
#get motif annotation file
tf_annon_df = pd.read_csv('../data/external/HOCOMOCOv11_annotation.csv',index_col=0)
tf_annon_df['id_trim'] = tf_annon_df['id'] + '.pwm.trim'
tf_name_to_id_dict = pd.Series(tf_annon_df.id_trim.values, index=tf_annon_df.tf.values).to_dict()
tf_id_to_name_dict = pd.Series(tf_annon_df.tf.values, index=tf_annon_df.id_trim.values).to_dict()
tf_name_to_id_abbr_dict = pd.Series(tf_annon_df.id.values, index=tf_annon_df.tf.values).to_dict()
tf_id_abbr_to_name_dict = pd.Series(tf_annon_df.tf.values, index=tf_annon_df.id.values).to_dict()


In [197]:
#get motif lengths
motif_to_len = {}
for file in sorted(glob.glob('../data/external/hocomoco_human_trim/*')):
    motif_id = os.path.basename(file).split('.pwm')[0]
#     print(tf_id_abbr_to_name_dict[motif_id])
    with open(file, 'r') as g:
        len_motif = len(g.readlines()[0].strip().split())
        motif_to_len[tf_id_abbr_to_name_dict[motif_id]] = len_motif

In [198]:
len(set(tf_id_abbr_to_name_dict.values())), len(motif_to_len)

(679, 678)

In [166]:
#read in motif enrichment results
ame_result_file = os.path.join(save_dir, 'motif_scan_neg_control_ame.tsv')
ame_result = pd.read_csv(ame_result_file, sep='\t')
ame_result['motif_DB'] = 'hocomoco_human_trim'
ame_result['motif'] = ame_result.motif_ID.map(tf_id_abbr_to_name_dict)
ame_result = ame_result[~ame_result.motif.isna()]
ame_result['in_vocab'] = ame_result.motif.isin(list_tfs)

In [167]:
ame_result

,rank,motif_DB,motif_ID,motif_alt_ID,consensus,p-value,adj_p-value,E-value,tests,FASTA_max,pos,neg,PWM_min,TP,%TP,FP,%FP,motif,in_vocab
0,1,hocomoco_human_trim,ZN423_HUMAN.H11MO.0.D,NaN,GRCACCCWAGGGTGC,7.200000e-17,4.100000e-15,3.150000e-12,57.0,89.0,89.0,1068.0,2.04,25.0,28.09,24.0,2.25,ZNF423,False
1,2,hocomoco_human_trim,EGR1_HUMAN.H11MO.0.A,NaN,GCGKGGGYGG,9.340000e-16,2.250000e-13,1.730000e-10,241.0,89.0,89.0,1068.0,7.79,27.0,30.34,36.0,3.37,EGR1,True
2,3,hocomoco_human_trim,KLF14_HUMAN.H11MO.0.D,NaN,GKGGGCGTG,3.930000e-15,2.790000e-13,2.150000e-10,71.0,89.0,89.0,1068.0,2.96,21.0,23.60,17.0,1.59,KLF14,False
3,4,hocomoco_human_trim,ZN547_HUMAN.H11MO.0.C,NaN,TGCTAATKCWGCARGCABRC,2.770000e-13,1.880000e-11,1.450000e-08,68.0,89.0,89.0,1068.0,9.19,17.0,19.10,11.0,1.03,ZNF547,False
4,5,hocomoco_human_trim,MAFF_HUMAN.H11MO.0.B,NaN,TGCTGASTCAGCABTTT,9.320000e-11,1.790000e-08,1.380000e-05,192.0,89.0,89.0,1068.0,6.37,21.0,23.60,36.0,3.37,MAFF,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,66,hocomoco_human_trim,SOX4_HUMAN.H11MO.0.B,NaN,YCTTTGTYY,3.100000e-05,1.050000e-02,8.070000e+00,340.0,89.0,89.0,1068.0,1.26,41.0,46.07,267.0,25.00,SOX4,False
66,67,hocomoco_human_trim,EPAS1_HUMAN.H11MO.0.B,NaN,ACGTG,1.120000e-04,1.100000e-02,8.430000e+00,98.0,89.0,89.0,1068.0,6.90,5.0,5.62,3.0,0.28,EPAS1,False
67,68,hocomoco_human_trim,SPIC_HUMAN.H11MO.0.D,NaN,RWGGG,1.650000e-05,1.130000e-02,8.680000e+00,690.0,89.0,89.0,1068.0,1.77,30.0,33.71,157.0,14.70,SPIC,False
68,69,hocomoco_human_trim,ZIC2_HUMAN.H11MO.0.D,NaN,CCTGCTGWG,3.450000e-05,1.170000e-02,9.000000e+00,341.0,89.0,89.0,1068.0,3.93,23.0,25.84,105.0,9.83,ZIC2,False


#### these are the motifs that are enriched in the negative control sequences

In [164]:
enriched_motifs_in_neg = sorted(ame_result.motif.unique())
print(len(enriched_motifs_in_neg))
print(enriched_motifs_in_neg)

70
['ATF1', 'BCL6', 'CBFB', 'DBP', 'E2F1', 'E2F3', 'E2F4', 'E2F6', 'E2F7', 'E2F8', 'EGR1', 'EGR2', 'EGR3', 'EGR4', 'EPAS1', 'ETS2', 'HLF', 'HOXB4', 'HSF1', 'HSF2', 'KLF1', 'KLF12', 'KLF14', 'KLF16', 'KLF3', 'KLF4', 'KLF5', 'KLF6', 'MAFF', 'MAFK', 'NKX2-2', 'NKX2-5', 'NRF1', 'PATZ1', 'POU3F1', 'PPARA', 'PRDM6', 'PURA', 'RARG', 'RHOXF1', 'RORC', 'RUNX3', 'SOX10', 'SOX4', 'SP1', 'SP2', 'SP4', 'SPDEF', 'SPIC', 'STAT4', 'STAT5A', 'STAT5B', 'TBX1', 'TFCP2', 'UBP1', 'XBP1', 'ZBTB17', 'ZBTB33', 'ZIC2', 'ZNF219', 'ZNF281', 'ZNF341', 'ZNF384', 'ZNF394', 'ZNF423', 'ZNF547', 'ZNF667', 'ZNF708', 'ZNF740', 'ZNF76']


#### these are the motifs that are enriched in the negative control sequences AND are part of vocab words

In [168]:
ame_result_filt = ame_result[ame_result.in_vocab]
print(ame_result.shape, ame_result_filt.shape)
enriched_motifs_in_neg_vocab = sorted(ame_result_filt.motif.unique())
print(len(enriched_motifs_in_neg_vocab))
print(enriched_motifs_in_neg_vocab)

(70, 19) (31, 19)
31
['CBFB', 'E2F1', 'E2F3', 'E2F4', 'E2F6', 'E2F7', 'EGR1', 'EGR2', 'EGR3', 'EGR4', 'KLF16', 'KLF4', 'KLF5', 'KLF6', 'MAFF', 'MAFK', 'NRF1', 'PATZ1', 'PURA', 'RARG', 'RUNX3', 'SOX10', 'SP1', 'SP2', 'SP4', 'TBX1', 'UBP1', 'XBP1', 'ZBTB17', 'ZNF219', 'ZNF394']


## AME motif enrichment negative controls (background vocab words)
`ame --verbose 1 --oc . --scoring avg --method fisher --hit-lo-fraction 0.25 --evalue-report-threshold 10.0 --control mpra_vocab_seq2.fasta mpra_neg_control_seq.fasta motifs.meme`

In [173]:
#read in motif enrichment results
ame_result_file = os.path.join(save_dir, 'motif_scan_neg_control_bckgdvocab_ame.tsv')
ame_result = pd.read_csv(ame_result_file, sep='\t')
ame_result['motif_DB'] = 'hocomoco_human_trim'
ame_result['motif'] = ame_result.motif_ID.map(tf_id_abbr_to_name_dict)
ame_result = ame_result[~ame_result.motif.isna()]
ame_result['in_vocab'] = ame_result.motif.isin(list_tfs)

print('enriched motifs')
enriched_motifs = sorted(ame_result.motif.unique())
print(len(enriched_motifs))
print(enriched_motifs)

print('enriched motifs with tested vocab pair')
ame_result_filt = ame_result[ame_result.in_vocab]
print(ame_result.shape, ame_result_filt.shape)
enriched_motifs_vocab = sorted(ame_result_filt.motif.unique())
print(len(enriched_motifs_vocab))
print(enriched_motifs_vocab)

enriched motifs
84
['ALX1', 'ATF1', 'BCL6', 'BPTF', 'CBFB', 'CDC5L', 'DBP', 'DLX2', 'DLX3', 'DLX5', 'DMBX1', 'EN2', 'FOXC2', 'FOXP3', 'GATA3', 'GATA5', 'GFI1B', 'GSX1', 'GSX2', 'HBP1', 'HLF', 'HMGA1', 'HOXA13', 'HOXA2', 'HOXB3', 'HOXB4', 'HOXB7', 'HOXB8', 'HOXC6', 'HOXC8', 'HOXD10', 'HOXD12', 'HOXD4', 'HOXD9', 'HSF1', 'HSF2', 'IRX3', 'LMX1A', 'LMX1B', 'MAFF', 'MAFK', 'MEIS1', 'MIXL1', 'MSX2', 'MYBL2', 'NFATC3', 'NFE2L1', 'NFIL3', 'NKX2-2', 'NKX2-5', 'NKX6-1', 'NKX6-2', 'ONECUT1', 'ONECUT2', 'PAX6', 'PBX2', 'POU1F1', 'POU3F1', 'POU3F3', 'PRDM6', 'PRRX1', 'PRRX2', 'RHOXF1', 'RORC', 'RUNX3', 'SMARCA1', 'SOX10', 'SOX2', 'SPDEF', 'STAT4', 'STAT5A', 'STAT5B', 'UBP1', 'ZBTB49', 'ZFHX3', 'ZNF146', 'ZNF250', 'ZNF333', 'ZNF384', 'ZNF394', 'ZNF423', 'ZNF547', 'ZNF667', 'ZNF680']
enriched motifs with tested vocab pair
(84, 19) (16, 19)
16
['CBFB', 'EN2', 'HBP1', 'HMGA1', 'IRX3', 'MAFF', 'MAFK', 'MSX2', 'MYBL2', 'NFE2L1', 'PRRX1', 'RUNX3', 'SOX10', 'UBP1', 'ZBTB49', 'ZNF394']


## AME motif enrichment for positive controls

`ame --verbose 1 --oc . --scoring avg --method fisher --hit-lo-fraction 0.25 --evalue-report-threshold 10.0 --control --shuffle-- --kmer 2 mpra_pos_control_seq.fasta motifs.meme`

AME motif enrichment neg control

motifs: '../data/external/hocomoco_human_trim_jaspar_format.txt'

In [174]:
#read in motif enrichment results
ame_result_file = os.path.join(save_dir, 'motif_scan_pos_control_ame.tsv')
ame_result = pd.read_csv(ame_result_file, sep='\t')
ame_result['motif_DB'] = 'hocomoco_human_trim'
ame_result['motif'] = ame_result.motif_ID.map(tf_id_abbr_to_name_dict)
ame_result = ame_result[~ame_result.motif.isna()]
ame_result['in_vocab'] = ame_result.motif.isin(list_tfs)

print('enriched motifs')
enriched_motifs = sorted(ame_result.motif.unique())
print(len(enriched_motifs))
print(enriched_motifs)

print('enriched motifs with tested vocab pair')
ame_result_filt = ame_result[ame_result.in_vocab]
print(ame_result.shape, ame_result_filt.shape)
enriched_motifs_vocab = sorted(ame_result_filt.motif.unique())
print(len(enriched_motifs_vocab))
print(enriched_motifs_vocab)

enriched motifs
19
['ATF1', 'CPEB1', 'CREB1', 'E2F7', 'EGR2', 'EGR4', 'ELK4', 'ERG', 'FOXL1', 'IRF3', 'IRF7', 'IRF8', 'NRF1', 'SP2', 'SP4', 'TAF1', 'VEZF1', 'ZBTB17', 'ZNF467']
enriched motifs with tested vocab pair
(20, 19) (12, 19)
11
['E2F7', 'EGR2', 'EGR4', 'FOXL1', 'IRF3', 'IRF8', 'NRF1', 'SP2', 'SP4', 'VEZF1', 'ZBTB17']


## AME motif enrichment for positive controls background vocab words

`ame --verbose 1 --oc . --scoring avg --method fisher --hit-lo-fraction 0.25 --evalue-report-threshold 10.0 --control mpra_vocab_seq2.fasta mpra_pos_control_seq.fasta motifs.meme`

AME motif enrichment neg control

motifs: '../data/external/hocomoco_human_trim_jaspar_format.txt'

In [175]:
#read in motif enrichment results
ame_result_file = os.path.join(save_dir, 'motif_scan_pos_control_bckgdvocab_ame.tsv')
ame_result = pd.read_csv(ame_result_file, sep='\t')
ame_result['motif_DB'] = 'hocomoco_human_trim'
ame_result['motif'] = ame_result.motif_ID.map(tf_id_abbr_to_name_dict)
ame_result = ame_result[~ame_result.motif.isna()]
ame_result['in_vocab'] = ame_result.motif.isin(list_tfs)

print('enriched motifs')
enriched_motifs = sorted(ame_result.motif.unique())
print(len(enriched_motifs))
print(enriched_motifs)

print('enriched motifs with tested vocab pair')
ame_result_filt = ame_result[ame_result.in_vocab]
print(ame_result.shape, ame_result_filt.shape)
enriched_motifs_vocab = sorted(ame_result_filt.motif.unique())
print(len(enriched_motifs_vocab))
print(enriched_motifs_vocab)

enriched motifs
1
['E2F5']
enriched motifs with tested vocab pair
(1, 19) (0, 19)
0
[]


## AME motif enrichment for vocab words

`ame --verbose 1 --oc . --scoring avg --method fisher --hit-lo-fraction 0.25 --evalue-report-threshold 10.0 --control --shuffle-- --kmer 2 mpra_vocab_seq2.fasta motifs.meme
`

AME motif enrichment neg control

motifs: '../data/external/hocomoco_human_trim_jaspar_format.txt'

In [177]:
#read in motif enrichment results
ame_result_file = os.path.join(save_dir, 'motif_scan_vocab_ame.tsv')
ame_result = pd.read_csv(ame_result_file, sep='\t')
ame_result['motif_DB'] = 'hocomoco_human_trim'
ame_result['motif'] = ame_result.motif_ID.map(tf_id_abbr_to_name_dict)
ame_result = ame_result[~ame_result.motif.isna()]
ame_result['in_vocab'] = ame_result.motif.isin(list_tfs)

print('enriched motifs')
enriched_motifs = sorted(ame_result.motif.unique())
print(len(enriched_motifs))
print(enriched_motifs)

print('enriched motifs with tested vocab pair')
ame_result_filt = ame_result[ame_result.in_vocab]
print(ame_result.shape, ame_result_filt.shape)
enriched_motifs_vocab = sorted(ame_result_filt.motif.unique())
print(len(enriched_motifs_vocab))
print(enriched_motifs_vocab)

enriched motifs
437
['AHR', 'AR', 'ARID3A', 'ARNT', 'ARNT2', 'ARNTL', 'ASCL1', 'ASCL2', 'ATF1', 'ATF2', 'ATF3', 'ATF4', 'ATF6', 'ATF7', 'ATOH1', 'BACH1', 'BACH2', 'BARHL2', 'BATF', 'BATF3', 'BCL11A', 'BCL6', 'BHLHA15', 'BHLHE23', 'BHLHE40', 'BHLHE41', 'BPTF', 'BSX', 'CBFB', 'CDC5L', 'CDX1', 'CDX2', 'CEBPA', 'CEBPB', 'CEBPZ', 'CLOCK', 'CPEB1', 'CREB1', 'CREB3', 'CREB3L1', 'CREB3L2', 'CREB5', 'CREM', 'CTCF', 'CTCFL', 'DLX2', 'DLX4', 'DLX5', 'DLX6', 'DMBX1', 'DMRT1', 'E2F1', 'E2F3', 'E2F4', 'E2F6', 'E2F7', 'E2F8', 'E4F1', 'EGR1', 'EGR2', 'EGR3', 'EGR4', 'EHF', 'ELF1', 'ELF2', 'ELF3', 'ELF5', 'ELK1', 'ELK3', 'ELK4', 'EN1', 'EPAS1', 'ERG', 'ESR2', 'ESRRA', 'ESRRB', 'ESX1', 'ETS1', 'ETS2', 'ETV1', 'ETV2', 'ETV3', 'ETV4', 'ETV5', 'ETV6', 'ETV7', 'FEV', 'FEZF1', 'FIGLA', 'FLI1', 'FOS', 'FOSB', 'FOSL1', 'FOSL2', 'FOXA1', 'FOXA2', 'FOXA3', 'FOXB1', 'FOXC1', 'FOXC2', 'FOXD1', 'FOXD2', 'FOXD3', 'FOXF1', 'FOXF2', 'FOXG1', 'FOXH1', 'FOXI1', 'FOXJ2', 'FOXJ3', 'FOXK1', 'FOXL1', 'FOXM1', 'FOXO1', 'FOXO

In [192]:
motif_df = pd.DataFrame({'motif':list(tf_id_abbr_to_name_dict.values())})
motif_df['len_motif'] = motif_df.motif.map(motif_to_len)
motif_df['in_vocab'] = motif_df.motif.isin(list_tfs)
motif_df['enriched_vocab'] = motif_df.motif.isin(enriched_motifs)
motif_df = motif_df[~motif_df.len_motif.isna()]

In [195]:
motif_df[motif_df.in_vocab & (~motif_df.enriched_vocab)]

,motif,len_motif,in_vocab,enriched_vocab
9,TFAP2B,9.0,True,False
10,TFAP2C,9.0,True,False
13,ARID5B,4.0,True,False
33,BARX2,4.0,True,False
62,EBF1,10.0,True,False
210,HBP1,5.0,True,False
226,EN2,5.0,True,False
288,IRX3,6.0,True,False
362,MYBL2,5.0,True,False
385,NFIA,5.0,True,False


In [240]:
pd.read_csv(os.path.join(save_dir, 'mpra_oligo_df_final_SUBMIT_091120.txt'),sep='\t').shape#oligo.str.len().value_counts()

(97210, 2)